In [1]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
import warnings
from operator import itemgetter
import yaml
import os
import yaml
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
# from src.vector_store_handler import VectorStoreHandler
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
# from langchain_core.parsers import StrOutputParser

In [2]:
# Suppress all warnings
warnings.filterwarnings('ignore')

# Example usage
def example_function():
    warnings.warn("This is a warning message", UserWarning)

example_function()  # This warning will not be displayed

In [9]:
class DocumentRetriever:
    def __init__(self, persist_directory):
        self.persist_directory = persist_directory
        self.embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={'device': 'mps'})
        self.vector_db = Chroma(persist_directory=self.persist_directory, embedding_function=self.embeddings)

    def retrieve_documents(self, query, top_k=5):
        # Retrieve top_k relevant document chunks based on the query
        return self.vector_db.similarity_search(query, k=top_k)

In [13]:


# class DocumentRetriever:
#     def __init__(self, persist_directory):
#         self.persist_directory = persist_directory
#         self.embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={'device': 'mps'})
#         self.vector_db = Chroma(persist_directory=self.persist_directory, embedding_function=self.embeddings)
#         self.retriever = self.vector_db.as_retriever()

#     def retrieve_documents(self, query, top_k=5):
#         # Retrieve top_k relevant document chunks based on the query
#         return self.retriever.similarity_search(query, k=top_k)

In [79]:
class RAGChatBot:
    def __init__(self, retriever, openai_api_key):
        self.retriever = retriever
        os.environ["OPENAI_API_KEY"] = openai_api_key
        self.chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
        self.prompt_template = hub.pull("rlm/rag-prompt")

    def format_docs(self, docs):
        return "\n\n".join(doc.page_content for doc in docs)

    def generate_response(self, user_query):
        # Retrieve documents relevant to the user query
        relevant_docs = self.retriever.retrieve_documents(user_query)
        relevant_context = self.format_docs(relevant_docs)
        # for relevant_doc in relevant_docs:
            # print(relevant_doc.metadata['source'])
            # print(relevant_doc.page_content)

        user_query = user_query + " Is the patient appropriate for inpatient or outpatient surgery? Why?"
#         instructions = """
#         1. You are a highly knowledgeable and empathetic medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
#         2. If a user chats about some topic on medical device, but doesn't ask/chat about inpatient or outpatient surgery, gently ask the user if you should help him regarding the inpatient vs outpatient surgery
#         3. If a user asks questions outside the medical domain, gently remind them that your expertise is focused on healthcare and surgical advice.
#         4. Chat with the user using the given context. If the context is not at all relevant to the question asked. Tell your reasoning behind your answer and let the user know that you are not sure of the answer, due to lack of relevent context on your end to answer the question.
#         5. If you feel that the data provided is insufficient to answer the question, gently ask the user about more information.
#   """
        instructions = """1. You are a highly knowledgeable medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure,based on the context you have. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
        2. You can save huge costs if you can correctly identify the case of Outpatient Surgery, so need not be conservative if you are confident, with the context you have"""
        # Construct the input data for the chain
        input_data = {
            "context": relevant_context,
            "question": user_query,  # Directly use the user query here
            "GUIDELINES": instructions
        }
        
        # Create a prompt using the template
        formatted_prompt = self.prompt_template.format(**input_data)
        
        # Generate a response using the LLM with invoke method
        response = self.chat_model.invoke(formatted_prompt)
        
        return response.content

    def generate_response_no_rag(self, question):
        template = """You are a highly knowledgeable and empathetic medical assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.

Guidelines:
1. **Focus on Medical Context:** Always center your responses around medical topics, particularly those related to surgical procedures and patient care settings.
2. **Inquire When Necessary:** If you require more information to make an informed decision, politely ask the user specific questions about the surgery details, such as the type of surgery, patient health status, or any existing medical conditions.
3. **Educational Approach:** Explain your suggestions clearly, using simple language that both healthcare professionals and patients can understand. Provide insights into the benefits and considerations of each option.
4. **Boundary Setting:** If a user asks questions outside the medical domain, gently remind them that your expertise is focused on healthcare and surgical advice.

Example Interaction:
- **User Query:** "I need to know if my knee surgery should be inpatient or outpatient."
- **Response:** "Knee surgeries can often be performed as outpatient procedures if the patient is in good health and the surgery is minimally invasive. However, if there are underlying health concerns or if a more complex procedure is needed, an inpatient setting might be safer. Could you provide more details about your specific situation?"

By adhering to these guidelines, you ensure that every interaction is informative, supportive, and within the scope of medical advisement.
        """
        system_message_prompt = SystemMessagePromptTemplate.from_template(template)
        human_template = "{text}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        chat_prompt = ChatPromptTemplate.from_messages(
            [system_message_prompt, human_message_prompt]
        )

        msg = self.chat_model.invoke(
            chat_prompt.format_prompt(
                text=question,
            ).to_messages()
        )

        return msg.content



    def generate_answer_with_chat_context(self, messages):
        """
        Generates a response with the last few messages and retrieved context using the chat model.

        Args:
            messages (list): List of dictionaries representing the conversation history.

        Returns:
            str: The generated response.
        """
        # Extract the last 4 messages plus the current one
        recent_messages = messages[-5:]  # Last 5 messages (4 historical + current)

        # Prepare input for retrieval (use only the content of messages, excluding the current one)
        retrieval_input = "\n".join(
            f"{message['role'].capitalize()}: {message['content']}" for message in recent_messages[:-1]
        )

        # Attempt to retrieve external documents
        try:
            retrieved_docs = self.retriever.retrieve_documents(retrieval_input)
            retrieved_context = self.format_docs(retrieved_docs)
        except Exception as e:
            retrieved_context = "No additional information could be retrieved."
            print(f"Error during document retrieval: {e}")

        # Ensure retrieved context is a valid string
        retrieved_context = str(retrieved_context)

        # Add a system message with the retrieved context
        system_message_prompt = """1. You are a highly knowledgeable medical chat assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate be conducted as an inpatient or outpatient procedure,based on the context you have. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
        2. You can save HUGE costs if you can correctly identify the case of Outpatient Surgery, so need not be conservative if you are confident, with the context you have.
        3. If the question/chat is outside the medical domain, gently remind your expertise."""
        system_message = SystemMessage(content= system_message_prompt + f"\n **RELEVANT INFORMATION RETREIVED**:\n{retrieved_context}")
        # system_message = SystemMessage(content="You are a Medical QA chat assistant and you are specialized in making decisions between Inpatient vs Outpatient surgeries.")

        # Convert recent conversation into LangChain message objects
        formatted_recent_messages = []
        for message in recent_messages:
            if message["role"] == "user":
                formatted_recent_messages.append(HumanMessage(content=message["content"]))
            elif message["role"] == "ai":
                formatted_recent_messages.append(AIMessage(content=message["content"]))
            else:
                raise ValueError(f"Unexpected role: {message['role']}")

        # Combine system message and formatted recent messages
        input_messages = [system_message] + formatted_recent_messages
        #formatted_recent_messages.append(HumanMessage(content="Outpatient Surgery saves a lot of money. So never be conservative until otherwise inpatient surgery is ABSOULETELY necessary."))
        # input_messages =  formatted_recent_messages
        # print(input_messages)

        # Generate the response using the chat model
        try:
            llm_response = self.chat_model.invoke(input_messages)
        except Exception as e:
            print(f"Error during model generation: {e}")
            raise

        # Parse the response if needed
        parsed_response = StrOutputParser().parse(llm_response)

        return parsed_response.content


    # def generate_answer_with_chat_context(self, messages):
    #     """
    #     Generates a response with the last few messages and retrieved context using the chat model.

    #     Args:
    #         messages (list): List of dictionaries representing the conversation history.

    #     Returns:
    #         str: The generated response.
    #     """
    #     # Extract the last 4 messages plus the current one
    #     recent_messages = messages[-5:]  # Last 5 messages (4 historical + current)

    #     # Prepare input for retrieval (use only the content of messages, excluding the current one)
    #     retrieval_input = "\n".join(
    #         message['content'] for message in recent_messages[:-1]
    #     )

    #     # Attempt to retrieve external documents
    #     try:
    #         retrieved_docs = self.retriever.retrieve_documents(retrieval_input)
    #         retrieved_context = self.format_docs(retrieved_docs)
    #     except Exception as e:
    #         retrieved_context = "No additional information could be retrieved."
    #         print(f"Error during document retrieval: {e}")

    #     # Ensure retrieved context is a valid string
    #     retrieved_context = str(retrieved_context)

    #     # Instructions for the assistant
    #     instructions = """1. You are a highly knowledgeable medical chat assistant specializing in advising on surgical procedures. Your primary role is to help both doctors and patients decide whether a given surgery is appropriate to be conducted as an inpatient or outpatient procedure based on the context you have. You provide detailed explanations to support your recommendations, ensuring that users make informed decisions.
    # 2. You can save huge costs if you can correctly identify the case of Outpatient Surgery, so need not be conservative if you are confident, with the context you have.
    # 3. If the question/chat is outside the medical domain, gently remind your expertise.
    # 4. If you are not sure, either ask questions to user or say you need more information or that you don't know."""

    #     # Construct input data for the LLM
    #     input_data = {
    #         "context": retrieved_context,
    #         "question": recent_messages[-1]['content'],  # Use the latest user query
    #         "GUIDELINES": instructions
    #     }
        
    #     # Create a prompt using your template
    #     formatted_prompt = self.prompt_template.format(**input_data)

    #     # Generate a response using the LLM with invoke method
    #     try:
    #         response = self.chat_model.invoke(formatted_prompt)
    #         parsed_response = StrOutputParser().parse(response)
    #     except Exception as e:
    #         print(f"Error during model generation: {e}")
    #         parsed_response = "An error occurred while generating a response."

    #     return parsed_response.content

In [80]:
# Configuration and setup
persist_directory = './chroma_db'
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialize retriever and chatbot
retriever = DocumentRetriever(persist_directory=persist_directory)
chat_bot = RAGChatBot(retriever=retriever, openai_api_key=openai_api_key)

In [81]:
messages = [
    # System Message (Optional but Recommended)
    # {
    #     "role": "system", 
    #     "content": "You are a helpful medical assistant specializing in surgical procedures and patient care."
    # },
    
    # Initial User Message
    {
        "role": "user", 
        "content": "“Patient is a 51 year old female who is planning to undergo laparoscopic myomectomy for fibroids. He has a history of hypertension, lupus, well-controlled asthma, and chronic renal insufficiency. She is compliant with her medications. There is a medical clearance note provided by her primary care provider. Help me choose inpatient vs outpatient surgery.”"
    }
    # ,
    # {
    #     "role": "ai",
    #     "content": "Based on the information provided, the patient is a 51-year-old female with a history of hypertension, lupus, well-controlled asthma, and chronic renal insufficiency. She is compliant with her medications, and there is a medical clearance note from her primary care provider. Given the patient's medical history, it is important to consider the potential risks and complications that may arise during and after the laparoscopic myomectomy surgery for fibroids. Patients with multiple comorbidities, such as hypertension, lupus, asthma, and chronic renal insufficiency, may be at a higher risk for postoperative complications. In this case, considering the patient's medical complexity and the potential need for close monitoring postoperatively, it would be advisable to recommend an inpatient surgery for the laparoscopic myomectomy. Inpatient surgery would allow for better monitoring of the patient's vital signs, pain management, and any potential complications that may arise due to her medical conditions. Additionally, inpatient surgery would provide the patient with access to immediate medical care and resources in case of any unforeseen issues during the recovery period. This approach prioritizes the patient's safety and well-being, ensuring that she receives the necessary care and attention postoperatively. Therefore, based on the patient's medical history and the complexity of her conditions, I would recommend that the laparoscopic myomectomy for fibroids be conducted as an inpatient procedure. This decision aims to optimize the patient's outcomes and minimize potential risks associated with her medical history."
    # },
    # {
    #     "role": "user",
    #     "content": "but there is medical clearence right?"}
    
   
]


In [82]:
print(chat_bot.generate_answer_with_chat_context(messages))

Based on the information provided, the patient is a 51-year-old female with multiple comorbidities, including hypertension, lupus, well-controlled asthma, and chronic renal insufficiency. She is compliant with her medications, and there is a medical clearance note from her primary care provider.

Given the complexity of the patient's medical history and the presence of multiple comorbidities, including chronic renal insufficiency, it is advisable to consider an inpatient surgery for the laparoscopic myomectomy for fibroids. Here's why:

1. **Comorbidities**: The patient's history of hypertension, lupus, asthma, and chronic renal insufficiency indicates a higher risk for potential complications during or after surgery. Inpatient surgery allows for closer monitoring of the patient's condition postoperatively.

2. **Medical Clearance**: While the patient has a medical clearance note, the presence of chronic conditions may require more intensive postoperative care, which is better provided

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a helpful assistant! Your name is Bob."
    ),
    HumanMessage(
        content="What is your name?"
    )
]

# Instantiate a chat model and invoke it with the messages
model = ChatOpenAI(model_name="gpt-3.5-turbo")
print(model.invoke(messages))

content='Hello! My name is Bob. How can I assist you today?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 27, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-b049debb-2abd-4c76-9374-c8ea6bf5a718-0'


Question 1

In [ ]:
user_query = """Patient is a 94 year old man who is scheduled for laparoscopic cholecystectomy. His BMI is 34. He has a history of end-stage renal disease on intermittent dialysis who receives it every Monday, Wednesday, and Saturday. In addition, he has a history of well-controlled hypertension and insulin-dependent diabetes.Should his surgery be inpatient or outpatient?"""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

In [49]:
user_query = """Patient is a 94 year old man who is scheduled for laparoscopic cholecystectomy. His BMI is 34. He has a history of end-stage renal disease on intermittent dialysis who receives it every Monday, Wednesday, and Saturday. In addition, he has a history of well-controlled hypertension and insulin-dependent diabetes.Should his surgery be inpatient or outpatient?"""
response = chat_bot.generate_response(user_query)
print(response)

The patient with a history of end-stage renal disease on intermittent dialysis should have inpatient surgery due to the higher complication rates associated with dialysis. Careful patient selection is crucial for patients with ESRD to ensure safe management in the ambulatory setting. Optimization of comorbid conditions and close monitoring are essential for these patients undergoing surgery.


In [35]:
# user_query = """hello can you help me with math?"""
# response = chat_bot.generate_response(user_query)
# print(response)

I don't know.


Question 2


In [33]:
user_query = """Patient is a 54 year old man who is scheduled for laparoscopic total hysterectomy. Her BMI is 63 and has a history of obstructive sleep apnea but not compliant with CPAP. She had a recent stroke 6 months ago and now on clopidogrel. In addition, she has a history of hypertension and hyperlipidemia, in which she takes amlodipine and a statin."""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Based on the patient's medical history and current health status, the decision between an inpatient or outpatient laparoscopic total hysterectomy should be carefully considered.

Given the patient's BMI of 63, history of obstructive sleep apnea, recent stroke, and current use of clopidogrel, there are several factors that suggest an inpatient procedure may be more appropriate in this case. Here's why:

1. **Obstructive Sleep Apnea (OSA):** Patients with OSA have an increased risk of complications during and after surgery, especially when under sedation. In an inpatient setting, the patient can be closely monitored postoperatively for any signs of respiratory distress.

2. **Recent Stroke and Clopidogrel Use:** The patient's history of stroke and current use of clopidogrel, an antiplatelet medication, increase the risk of bleeding during surgery. In an inpatient setting, the medical team can closely monitor the patient's bleeding parameters and manage any potential complications effecti

In [51]:
user_query = """Patient is a 54 year old man who is scheduled for laparoscopic total hysterectomy. Her BMI is 63 and has a history of obstructive sleep apnea but not compliant with CPAP. She had a recent stroke 6 months ago and now on clopidogrel. In addition, she has a history of hypertension and hyperlipidemia, in which she takes amlodipine and a statin."""
response = chat_bot.generate_response(user_query)

print(response)

The patient with a BMI of 63, history of obstructive sleep apnea, recent stroke, and on clopidogrel may not be appropriate for outpatient surgery due to the increased risk of perioperative complications associated with obesity and OSA. The presence of comorbidities and the need for advanced treatment in case of complications should be considered when deciding on the appropriate surgical venue. Further assessment of the patient's overall health and risk factors is necessary to determine the safest approach for the surgery.


In [37]:
user_query = """Patient is a 26 year old female who is scheduled for an arthroscopic rotator cuff repair. As a child, he had congenital heart disease, which was described as a ventral septal defect in his heart. This was repaired in childhood. Today, he is an active soccer player and has not had any recent issues with his cardiac problems."""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Given the patient's history of congenital heart disease with a ventral septal defect repair in childhood, it is crucial to consider this condition when deciding whether the arthroscopic rotator cuff repair should be an inpatient or outpatient procedure.

In this case, due to the patient's past cardiac history, it would be advisable to recommend an inpatient procedure for the arthroscopic rotator cuff repair. This choice allows for closer monitoring of the patient's vital signs, including heart function, during and after the surgery. Inpatient care provides immediate access to medical professionals and resources in case any unforeseen complications arise, especially considering the patient's prior heart condition.

Additionally, an inpatient setting ensures that the patient can receive appropriate post-operative care and monitoring to address any potential cardiac concerns that may arise due to the stress of surgery and anesthesia.

It is essential to prioritize the patient's safety and

In [52]:
user_query = """Patient is a 26 year old female who is scheduled for an arthroscopic rotator cuff repair. As a child, he had congenital heart disease, which was described as a ventral septal defect in his heart. This was repaired in childhood. Today, he is an active soccer player and has not had any recent issues with his cardiac problems."""
response = chat_bot.generate_response(user_query)
print(response)

The patient is not appropriate for outpatient surgery due to a history of congenital heart disease and repaired ventral septal defect. The patient may have a left ventricle ejection fraction (LVEF) of <35% or New York Heart Association class >II, making them unsuitable for outpatient surgery. Inadequate support and lack of personnel in an outpatient setting may not be equipped to manage potential perioperative complications related to cardiac issues.


In [39]:
user_query = """Patient is a 83 year old man who is scheduled for a ureteral stent placement for kidney stones. His BMI is 42. He has a history of well-controlled COPD, hypertension, obstructive sleep apnea on CPAP, and coronary artery disease. He had a myocardial infarction 5 years ago, in which he underwent a CABG. Since then, he has not had any episodes of myocardial ischemia."""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Given the patient's medical history and current health status, the decision between an inpatient or outpatient ureteral stent placement procedure is crucial. Let's break down the considerations:

1. **Ureteral Stent Placement Procedure:** This procedure involves placing a small tube (stent) in the ureter to help urine flow from the kidney to the bladder. It is commonly done to treat kidney stones or other urinary obstructions.

2. **Patient's Age and BMI:** The patient is 83 years old with a BMI of 42, which may increase the risk of complications during and after the procedure, such as wound healing issues and infections.

3. **Medical History:** The patient has a history of COPD, hypertension, obstructive sleep apnea, and coronary artery disease with a previous CABG. These conditions, especially COPD and coronary artery disease, may impact the patient's ability to tolerate the procedure and anesthesia.

4. **Myocardial Infarction History:** The patient had a myocardial infarction 5 ye

In [40]:
user_query = """Patient is a 83 year old man who is scheduled for a ureteral stent placement for kidney stones. His BMI is 42. He has a history of well-controlled COPD, hypertension, obstructive sleep apnea on CPAP, and coronary artery disease. He had a myocardial infarction 5 years ago, in which he underwent a CABG. Since then, he has not had any episodes of myocardial ischemia."""
response = chat_bot.generate_response(user_query)
print(response)

The patient with a BMI of 42 and a history of well-controlled COPD, hypertension, obstructive sleep apnea, and coronary artery disease may not be appropriate for outpatient surgery due to the risk of perioperative cardiac complications. Severe obesity alone can lead to cardiomyopathy even without intrinsic cardiac disease, increasing the risk of cardiovascular complications. Preoperative cardiac assessment is critical in this patient population.


In [41]:
user_query = """Patient is a 72 year old female who is scheduled for total shoulder arthroplasty. She has a history of ischemic cardiomyopathy and hypertension. Her last echo demonstrated an ejection of fraction of 25% with severe mitral stenosis. She is unable to walk up 2 flights of stairs without feeling short of breath."""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Based on the patient's medical history and current health status, it is recommended that the total shoulder arthroplasty be conducted as an inpatient procedure. Here's why:

1. **Complexity of the Surgery:** Total shoulder arthroplasty is a significant surgical procedure that involves replacing the damaged parts of the shoulder joint with artificial components. Given the patient's age and medical conditions, including ischemic cardiomyopathy and severe mitral stenosis, there may be an increased risk of complications during and after the surgery.

2. **Cardiac Health Concerns:** The patient's history of ischemic cardiomyopathy, hypertension, and a low ejection fraction of 25% indicate a compromised heart function. An inpatient setting allows for closer monitoring of the patient's cardiac status throughout the surgery and during the initial recovery period.

3. **Functional Limitations:** The patient's difficulty in walking up stairs without feeling short of breath suggests limited funct

In [58]:
user_query = """Patient is a 72 year old female who is scheduled for total shoulder arthroplasty. She has a history of ischemic cardiomyopathy and hypertension. Her last echo demonstrated an ejection of fraction of 25% with severe mitral stenosis. She is unable to walk up 2 flights of stairs without feeling short of breath."""
response = chat_bot.generate_response(user_query)
print(response)

The patient is not appropriate for outpatient surgery due to severe mitral stenosis and symptoms of poor exercise tolerance and dyspnea. Patients with echocardiographic signs of severe valvular disease and symptoms are not suitable candidates for ambulatory surgery. The patient should be considered for inpatient surgery.


In [48]:
user_query = """Patient is a 84 year old male with an ASA classification score of 4, who is scheduled for a laparoscopic inguinal hernia repair. He has smoked 100 pack years and has severe COPD requiring continuous oxygen. In addition, he has end-stage renal disease and is awaiting dialysis placement."""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Based on the information provided, the patient is an 84-year-old male with multiple significant health issues, including a high ASA classification score of 4, severe COPD, end-stage renal disease, and a history of heavy smoking. Given these complex medical conditions, the decision between an inpatient or outpatient setting for the laparoscopic inguinal hernia repair is crucial.

In this case, considering the patient's age, ASA score, severe COPD requiring continuous oxygen, and end-stage renal disease, it is highly recommended that the laparoscopic inguinal hernia repair be performed as an inpatient procedure. Here's why:

1. **Monitoring and Management:** In an inpatient setting, the patient can be closely monitored post-surgery for any potential complications that may arise due to his multiple health issues. Continuous oxygen support and specialized care for COPD and renal disease can be provided effectively in a hospital environment.

2. **Risk of Complications:** The patient's hist

In [71]:
user_query = """Patient is a 84 year old male with an ASA classification score of 4, who is scheduled for a laparoscopic inguinal hernia repair. He has smoked 100 pack years and has severe COPD requiring continuous oxygen. In addition, he has end-stage renal disease and is awaiting dialysis placement."""
response = chat_bot.generate_response(user_query)
print(response)

The patient is not appropriate for outpatient surgery due to severe COPD requiring continuous oxygen and end-stage renal disease awaiting dialysis placement. Patients with severely diminished forced expiratory volume in 1 second are at risk for respiratory complications and are not good candidates for outpatient surgery. Factors such as patient positioning challenges and need for specialized equipment should be considered before scheduling in an ambulatory setting.


In [50]:
user_query ="""Patient is a 63 year old man who is planning to undergo total knee arthroplasty. He had a history of multiple episodes of ventricular fibrillation and eventually underwent placement of a AICD one year ago. He is unable to walk up two flights of stairs without feeling chest pain. In addition, he has a history of diabetes, hypertension, and obstructive sleep apnea.
"""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Given the patient's complex medical history, including a previous history of ventricular fibrillation, placement of an Automatic Implantable Cardioverter Defibrillator (AICD), chest pain with exertion, diabetes, hypertension, and obstructive sleep apnea, it is recommended that the total knee arthroplasty be performed as an inpatient procedure.

Here's why:
1. **Cardiac History:** The patient's history of ventricular fibrillation and AICD placement indicates a significant cardiac condition. An inpatient setting allows for closer monitoring of the patient's cardiac status during and after the surgery.
   
2. **Chest Pain:** The chest pain experienced by the patient with exertion could be related to underlying cardiac issues. In an inpatient setting, immediate access to cardiac monitoring and interventions is available if needed.

3. **Multiple Comorbidities:** The presence of diabetes, hypertension, and obstructive sleep apnea increases the risk of complications during and after surgery.

In [60]:
user_query ="""Patient is a 63 year old man who is planning to undergo total knee arthroplasty. He had a history of multiple episodes of ventricular fibrillation and eventually underwent placement of a AICD one year ago. He is unable to walk up two flights of stairs without feeling chest pain. In addition, he has a history of diabetes, hypertension, and obstructive sleep apnea.
"""
response = chat_bot.generate_response(user_query)
print(response)

The patient with a history of multiple episodes of ventricular fibrillation, diabetes, hypertension, and obstructive sleep apnea may not be appropriate for outpatient surgery due to the increased risk factors and potential complications. Patients with major cardiac issues and poorly controlled diabetes are typically excluded from outpatient arthroplasty to ensure optimal safety and outcomes. It is important to carefully consider the patient's overall health status and risk factors before determining the appropriate setting for surgery.


In [52]:
user_query ="""Patient is a 51 year old female who is planning to undergo laparoscopic myomectomy for fibroids. He has a history of hypertension, lupus, well-controlled asthma, and chronic renal insufficiency. She is compliant with her medications. There is a medical clearance note provided by her primary care provider.
"""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Given the patient's medical history of hypertension, lupus, well-controlled asthma, and chronic renal insufficiency, it is important to consider these factors when deciding whether the laparoscopic myomectomy should be performed as an inpatient or outpatient procedure.

In this case, due to the presence of multiple medical conditions, it would be advisable for the surgery to be conducted as an inpatient procedure. Here's why:

1. **Monitoring and Management:** In an inpatient setting, the patient can be closely monitored post-surgery for any potential complications related to her medical conditions. This allows for prompt intervention if needed.

2. **Pain Management:** Given the nature of the surgery and the patient's medical history, adequate pain management and monitoring can be better achieved in an inpatient setting.

3. **Medical Support:** Inpatient care provides access to a multidisciplinary team of healthcare professionals who can address any medical concerns that may arise du

In [19]:
user_query ="""Patient is a 51 year old female who is planning to undergo laparoscopic myomectomy for fibroids. She has a history of hypertension, lupus, well-controlled asthma, and chronic renal insufficiency. She is compliant with her medications. There is a medical clearance note provided by her primary care provider.
"""
response = chat_bot.generate_response(user_query)
print(response)

The patient is appropriate for outpatient surgery due to the well-controlled comorbid conditions, compliance with medications, and medical clearance note provided by her primary care provider. Careful patient selection is essential for safe management in the ambulatory setting, considering factors such as chronic renal insufficiency and other comorbidities. The decision for inpatient or outpatient surgery should be made based on the patient's overall health status and the specific procedure being performed.


In [54]:
user_query ="""Patient is a 64 year old man who is planning to undergo an umbilical hernia repair. He has a history of ischemic cardiomyopathy (with recent ejection fraction of 30%), cirrhosis with persistent ascites, end-stage renal disease on dialysis, moderate pulmonary hypertension, and moderate aortic stenosis. He had a myocardial infarction 5 months, in which is received a drug eluting stent to his LAD. He is now on dual anti-platelet therapy.
"""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Given the complex medical history of the patient, it is advisable for the umbilical hernia repair to be performed as an inpatient procedure rather than outpatient. Here's why:

1. **Multiple Comorbidities:** The patient's history of ischemic cardiomyopathy, cirrhosis, end-stage renal disease, pulmonary hypertension, aortic stenosis, and recent myocardial infarction with a drug-eluting stent places him at high risk for complications during and after surgery. These conditions may require close monitoring and specialized care that are more readily available in an inpatient setting.

2. **Medication Management:** The patient is currently on dual anti-platelet therapy following the placement of a drug-eluting stent. In an inpatient setting, healthcare providers can closely monitor and manage the patient's medications to prevent any interactions or adverse effects during and after the surgery.

3. **Postoperative Care:** Given the patient's compromised health status, he may require more inte

In [18]:
user_query ="""Patient is a 64 year old man who is planning to undergo an umbilical hernia repair. He has a history of ischemic cardiomyopathy (with recent ejection fraction of 30%), cirrhosis with persistent ascites, end-stage renal disease on dialysis, moderate pulmonary hypertension, and moderate aortic stenosis. He had a myocardial infarction 5 months, in which is received a drug eluting stent to his LAD. He is now on dual anti-platelet therapy.
"""
response = chat_bot.generate_response(user_query)
print(response)

The patient is not appropriate for outpatient surgery due to his comorbidities, including moderate aortic stenosis and recent myocardial infarction with a drug-eluting stent. The presence of these conditions necessitates careful monitoring and management, making inpatient surgery the more suitable option. Discontinuation of dual anti-platelet therapy and the need for specialized care in the postoperative period are factors that support the decision for inpatient surgery.


In [56]:
user_query ="""Patient is a 98 year old female who is planning to undergo total thyroidectomy. He has a BMI of 27 and has a history of hypertension, diabetes, hyperlipidemia, and depression. His last hemoglobin A1c was 7.2.
"""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Given the patient's age of 98 and the presence of multiple medical conditions such as hypertension, diabetes, hyperlipidemia, and depression, along with a BMI of 27, the decision between inpatient and outpatient total thyroidectomy should be carefully considered.

In this case, due to the patient's age and medical history, it is advisable to recommend an inpatient procedure for the total thyroidectomy. Here's why:

1. **Postoperative Monitoring:** An inpatient setting allows for closer monitoring of the patient's vital signs, pain management, and potential complications that may arise after surgery, especially considering the patient's age and existing medical conditions.

2. **Medical Support:** Inpatient care provides access to medical professionals round-the-clock, ensuring immediate intervention in case of any unforeseen issues or emergencies.

3. **Recovery and Rehabilitation:** The patient may benefit from a more structured and supervised recovery process in an inpatient setting,

In [17]:
user_query ="""Patient is a 98 year old female who is planning to undergo total thyroidectomy. He has a BMI of 27 and has a history of hypertension, diabetes, hyperlipidemia, and depression. His last hemoglobin A1c was 7.2.
"""
response = chat_bot.generate_response(user_query)
print(response)

The patient with a history of hypertension, diabetes, hyperlipidemia, and depression, with a BMI of 27 and a last hemoglobin A1c of 7.2, may be appropriate for outpatient surgery based on their long-term blood glucose control. The ambulatory setting may provide advantages in managing diabetes with minimal disruption of oral intake and quick resumption of home medications. However, the decision should be made in consultation with the treating endocrinologist to determine safe infusion rates and perioperative treatment goals.


In [21]:
user_query ="Hello"
response = chat_bot.generate_response(user_query)
print(response)

The patient should be categorized based on the severity of their comorbidities. If the comorbidities are severe and unstable, inpatient surgery is recommended. If the comorbidities can be optimized, elective surgery can be performed in the outpatient setting after optimization. The decision should be made based on the patient's overall health status and stability.


In [ ]:
user_query = """Patient is a 84 year old male with an ASA classification score of 4, who is scheduled for a laparoscopic inguinal hernia repair. He has smoked 100 pack years and has severe COPD requiring continuous oxygen. In addition, he has end-stage renal disease and is awaiting dialysis placement."""
response = chat_bot.generate_response_no_rag(user_query)
print(response)

Based on the information provided, the patient is an 84-year-old male with multiple significant health issues, including a high ASA classification score of 4, severe COPD, end-stage renal disease, and a history of heavy smoking. Given these complex medical conditions, the decision between an inpatient or outpatient setting for the laparoscopic inguinal hernia repair is crucial.

In this case, considering the patient's age, ASA score, severe COPD requiring continuous oxygen, and end-stage renal disease, it is highly recommended that the laparoscopic inguinal hernia repair be performed as an inpatient procedure. Here's why:

1. **Monitoring and Management:** In an inpatient setting, the patient can be closely monitored post-surgery for any potential complications that may arise due to his multiple health issues. Continuous oxygen support and specialized care for COPD and renal disease can be provided effectively in a hospital environment.

2. **Risk of Complications:** The patient's hist